In [3]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *
from preprocess_utilities import *
from cell_utilities import *

Setting environment for AWS compute node
rm -rf /shared/CSHL_data_processed/MD585/MD585_anchor.txt && mkdir -p /shared/CSHL_data_processed/MD585
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_anchor.txt /shared/CSHL_data_processed/MD585/MD585_anchor.txt


No vtk
Child returned 0
Child returned 0
0.42 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD585
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_sorted_filenames.txt /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt


Child returned 0
0.41 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD585/MD585_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD585
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD585/MD585_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropbox.txt /shared/CSHL_data_processed/MD585/MD585_alignedTo_MD585-N47-2015.07.16-22.50.52_MD585_3_0141_cropbox.txt


...

Child returned 0
0.39 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD658/MD658_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD658
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD658/MD658_sorted_filenames.txt /shared/CSHL_data_processed/MD658/MD658_sorted_filenames.txt


Child returned 0
0.45 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD658
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropbox.txt /shared/CSHL_data_processed/MD658/MD658_alignedTo_MD658-N58-2017.03.31-19.59.31_MD658_2_0173_cropbox.txt


Child returned 0
0.47 seconds.


In [1]:
# run_distributed(command='sudo apt-get install -y \
#   build-essential \
#   cython \
#   libmysqlclient-dev \
#   libhdf5-dev \
#   libxml2-dev \
#   libxslt1-dev \
#   openjdk-8-jdk \
#   python-dev \
#     python-h5py \
#     python-mysqldb \
#     python-vigra \
#   python-wxgtk3.0 \
#   python-zmq',
#                     argument_type='single',
#                    cluster_size=16)

# wait_qsub_complete()


In [2]:
# run_distributed(command='cd /shared/CellProfiler/; sudo pip install --editable .',
#                     argument_type='single',
#                    cluster_size=16)

# wait_qsub_complete()

In [4]:
download_from_s3(CELLPROFILER_PIPELINE_FP)

rm -rf /shared/CSHL_cells_v2/SegmentCells.cppipe && mkdir -p /shared/CSHL_cells_v2
aws s3 cp s3://mousebrainatlas-data/CSHL_cells_v2/SegmentCells.cppipe /shared/CSHL_cells_v2/SegmentCells.cppipe


Child returned 0
Child returned 0
1.68 seconds.


# Whole pipeline

In [54]:
valid_filenames[first_idx_among_valid:last_idx_among_valid+1][116]

'MD585-IHC36-2015.08.19-13.58.57_MD585_1_0106'

In [43]:
# for stack in all_nissl_stacks:
for stack in ['MD585']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    # Detect cells.
    
    t = time.time()
    sys.stderr.write('Detect cells using Cell Profiler ...')

    script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells_cellprofiler_v2.py')
    
    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1300 sec / stack

    run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()
    
    # Generate cell auxiliary data.

    t = time.time()
    sys.stderr.write('Detect cells ...')

    script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells.py')
    
    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 271 sec / stack
    
    run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()

rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 0
Child returned 1
0.34 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.34 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Detect cells using Cell Profiler ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


...

rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 719.665955 seconds
Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.


KeyboardInterrupt: 

# Step 1: Run cellprofiler

In [30]:
script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells_cellprofiler_v2.py')

In [31]:
# for stack in all_nissl_stacks:
for stack in ['MD594']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    t = time.time()
    sys.stderr.write('Detect cells using Cell Profiler ...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # 16 or 32 or 64 sections, 420 seconds
    # 128, 568
    # all 1300 seconds.

Detect cells using Cell Profiler ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1242.701211 seconds


In [37]:
run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
wait_qsub_complete()

rm -f ~/stderr_*; rm -f ~/stdout_*


Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.


# Step 2: Generate derived data

In [38]:
script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells.py')

In [39]:
# for stack in all_nissl_stacks:
for stack in ['MD585']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    t = time.time()
    sys.stderr.write('Detect cells ...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1][116:117]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 271 sec / stack

Detect cells ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 271.996746 seconds


In [56]:
run_distributed(command='rm -r /scratch/*',
                        argument_type='single',
                       cluster_size=16)
wait_qsub_complete()

rm -f ~/stderr_*; rm -f ~/stdout_*


Child returned 0
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
